# Objetos y Modulos

## Definición a la API pública

https://platzi.com/clases/1378-python-practico/14182-definicion-a-la-api-publica/

Dejaremos nuestro archivo *main.py* quieto. Solo añadiremos una linea, para indicarle se ejecute python de manera implicita:

        #!/usr/bin/env python

Hasta ahora hemos trabajado en un solo archivo. Sin embargo en la industria, es comun usar muchos archivos de tal forma que el software sea legible, donde cada uno de los archivos tendra una funcione especifica

Crearemos un nuevo directorio: **app-platzi-ventas**. Y crearemos una estructura de archivos:

- pv.py: es el punto de entrada

![](https://i.imgur.com/2mBSbUL.png)

En Python, un subdirectorio, verbigracia *clients* se conoce como **modulos**.

### ¿Porque Instalar una aplicacion?

Generalmente cuando usas cualquier aplicacion de linea de comandos, no tienes que colocar:

    python3 git logs ❌

Simplemente ejecutamos el comando y ya. Lo que queremos es poder ejecutar nuestra aplicacion simplemente asi:

    pv

Nuestra aplicacion se llama **pv**(platzi ventas).

#### Empezaremos por el archivo pv.py 

Aqui solo ira el punto de entrada


![](https://i.imgur.com/dcMgdRT.png)


- Para registrar los comandos, donde *cli* hace referencia a la funcion,(da un poco de legibilidad al codigo, de otra forma escribiriamos *client_commands.clients*) :

    cli.add_command(client_commands.all)

- Se importa del modulo clients *commands* que se define a continuacion

### Definiendo comandos basicos

Por ahora, solo esta el comando *search*. 

![](https://i.imgur.com/P980C2p.png)

- all es un alias de clientes -- all apunta a la funcion clients


### Implementar setup.py

Para invocar directamente la *App* de linea de comandos, sin tener que llamar al interprete de python.

- setuptools es una libreria que tiene muchas funcionalidades, pero solo usaremos el modulo *setup* que parece bastante complejo

*"Setuptools is a collection of enhancements to the Python distutils that allow developers to more easily build and distribute Python packages, especially ones that have dependencies on other packages."*


![](https://i.imgur.com/VFT4iEY.png)

#### Instalacion de la App

- EN un ambiente virtual
- Instalarla, no olvidar el puntico. La opcion **--editable** significa los cambios que se hagan se veran reflejados, asi este instalada. 

        pip install --editable .

![](https://i.imgur.com/R9I7RBV.png)

verificar con:

    which pv

Salida:

    https://i.imgur.com/R9I7RBV.png    

#### Verificando las funciones de ayuda

![](https://i.imgur.com/HbSK7jn.png)

        

## Pasando el contexto

Desafortunadamente la explicacion fue muy deficiente, y la documentacion es algon enredada. Asi que hare mi mejor esfuerzo:

En **pv.py**: 

- Declararemos una constante con el nombre del archivo:

        CLIENTS_TABLE = '.clients.csv'

- Se hace uso del decorador @click.pass_context, que al usarlo devuelve un objeto contexto(ctx), Que inicializamos como un diccionario vacio.

- Asignaremos un campo llave:valor al contexto con el nombre del archivo 

        ctx.obj['clients_table'] = CLIENTS_TABLE

![](https://i.imgur.com/3rPHbUs.png)


En **commands.py**

![](https://i.imgur.com/ELuTorJ.png)

- con el decorador @click.pass_context le pasamos el contexto.
- asignamos esta variable y miramos que hay dentro de ella:

        client_service = ctx.obj['clients_table']

- ejecutamos la aplicacion:

        pv clients list 
![](https://i.imgur.com/5BrC7DX.png)

No es nada raro. Solo es una referencia al archivo desde donde se van a leer los datos. 😊

## Primera implementacion del comando list

Todavia no se hara uso de POO. Se hara desde *commands.py*
- Agregando... 

![](https://i.imgur.com/HHViXaG.png)

- Solo cambiaremos por ahora el nombre de client_service a client_table
- Se agrega la funcion _load_data_from_csv como en main.py
- Se hace uso de la libreria *tabulate* para mostrarlo en forma de tabla 

![](https://i.imgur.com/h6RuPxx.png)

Asi ejecutamos:

    pv clients list

![](https://i.imgur.com/W2cS3y4.png)
